In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import gc
import glob

# 그래프 기본 테마 설정
# https://coldbrown.co.kr/2023/07/%ED%8C%8C%EC%9D%B4%EC%8D%AC-%EC%8B%A4%EC%A0%84%ED%8E%B8-08-seaborn-sns-set%EC%9D%84-%ED%86%B5%ED%95%B4-%EC%8A%A4%ED%83%80%EC%9D%BC-%EC%84%A4%EC%A0%95%ED%95%98%EA%B8%B0/
sns.set()

# 그래프 기본 설정
plt.rcParams['font.family'] = 'Malgun Gothic'
# plt.rcParams['font.family'] = 'AppleGothic'
plt.rcParams['figure.figsize'] = 12, 6
plt.rcParams['font.size'] = 14
plt.rcParams['axes.unicode_minus'] = False


# 복잡한 통계 처리를 위한 라이브러리
from scipy import stats

In [2]:
data_type = "test"
# month = "07"
# category = "청구입금정보"

# local
root_path = '../data/open'

# colab
# root_path = '/content/drive/MyDrive/12조 파이널프로젝트/data'

drive_folder = f'{root_path}/{data_type}/4.청구입금정보/'

In [5]:
# parquet 파일 데이터를 읽어온다.
billing = pd.read_parquet(f'{drive_folder}test_청구정보_통합.parquet')
billing

,기준년월,ID,대표결제일,대표결제방법코드,대표청구지고객주소구분코드,대표청구서수령지구분코드,청구서수령방법,청구서발송여부_B0,청구서발송여부_R3M,청구서발송여부_R6M,...,할인금액_청구서_B0M,상환개월수_결제일_R6M,상환개월수_결제일_R3M,선결제건수_R6M,선결제건수_R3M,연체건수_R6M,연체건수_R3M,혜택수혜금액_R3M,포인트_마일리지_환산_B0M,혜택수혜금액
0,201807,TEST_00000,25,자동이체,미확인,이메일,이메일,1,1,1,...,0,5,2,0,0,0,0,143,148,56
1,201807,TEST_00001,13,자동이체,미확인,당사페이앱+이메일,문자메세지,1,1,1,...,14,6,3,0,0,0,0,0,0,0
2,201807,TEST_00002,25,자동이체,미확인,이메일,이메일,1,1,1,...,50,6,3,0,0,0,0,148,0,50
3,201807,TEST_00003,25,자동이체,주거지,우편,우편,1,1,1,...,0,2,1,0,0,4,2,3,0,1
4,201807,TEST_00004,27,자동이체,주거지,우편,우편,1,1,1,...,0,6,3,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
599995,201812,TEST_99995,20,자동이체,회사,우편,우편,0,0,0,...,0,0,0,0,0,0,0,0,0,0
599996,201812,TEST_99996,20,자동이체,주거지,우편,우편,1,1,1,...,50,5,3,3,2,0,0,149,0,49
599997,201812,TEST_99997,15,자동이체,미확인,이메일,이메일,0,0,0,...,0,0,0,0,0,0,0,0,0,0
599998,201812,TEST_99998,12,자동이체,미확인,K톡명세서+이메일,K톡,1,1,1,...,242,1,1,7,3,0,0,0,0,0


In [9]:
# 메모리 사용량이 너무 많아서, int64를 int32로 바꾼다.
for col in billing.select_dtypes(include='int64').columns:
    billing[col] = billing[col].astype('int32')

In [13]:
# 🔎 결측치 확인
# print("🔍 [결측치 개수]")
# print("▶ billing:\n", billing.isnull().sum().sort_values(ascending=False))

In [17]:
# 🔎 데이터 타입
# print("▶ billing dtypes\n", billing.dtypes)

In [21]:
# 🔎 빈 문자열 확인
# print("\n🔍 [빈 문자열 개수]")
# print("▶ billing 빈 문자열:\n", (billing == '').sum())

In [23]:
billing['대표결제방법코드'].unique()

array(['자동이체'], dtype=object)

In [25]:
billing.drop(columns=['대표결제방법코드'], inplace=True)

In [27]:
# csv너무 느려서 parquet으로 다시 . .
billing.to_parquet(f'{drive_folder}test_청구정보_통합_전처리1.parquet', index=False)

In [29]:
# 잔액정보 (모든 월 통합) 통합 parquet
billing_eda = pd.read_parquet(f'{drive_folder}test_청구정보_통합_전처리1.parquet')
billing_eda

,기준년월,ID,대표결제일,대표청구지고객주소구분코드,대표청구서수령지구분코드,청구서수령방법,청구서발송여부_B0,청구서발송여부_R3M,청구서발송여부_R6M,청구금액_B0,...,할인금액_청구서_B0M,상환개월수_결제일_R6M,상환개월수_결제일_R3M,선결제건수_R6M,선결제건수_R3M,연체건수_R6M,연체건수_R3M,혜택수혜금액_R3M,포인트_마일리지_환산_B0M,혜택수혜금액
0,201807,TEST_00000,25,미확인,이메일,이메일,1,1,1,4931,...,0,5,2,0,0,0,0,143,148,56
1,201807,TEST_00001,13,미확인,당사페이앱+이메일,문자메세지,1,1,1,10152,...,14,6,3,0,0,0,0,0,0,0
2,201807,TEST_00002,25,미확인,이메일,이메일,1,1,1,13223,...,50,6,3,0,0,0,0,148,0,50
3,201807,TEST_00003,25,주거지,우편,우편,1,1,1,2112,...,0,2,1,0,0,4,2,3,0,1
4,201807,TEST_00004,27,주거지,우편,우편,1,1,1,4406,...,0,6,3,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
599995,201812,TEST_99995,20,회사,우편,우편,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
599996,201812,TEST_99996,20,주거지,우편,우편,1,1,1,359,...,50,5,3,3,2,0,0,149,0,49
599997,201812,TEST_99997,15,미확인,이메일,이메일,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
599998,201812,TEST_99998,12,미확인,K톡명세서+이메일,K톡,1,1,1,21273,...,242,1,1,7,3,0,0,0,0,0
